In [9]:
# analysis notebook 

# see opt_fun.py for how these are calculated 
# see the individual files for the data 

In [10]:
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from IPython.display import display
import numpy as np

In [11]:
from analysis_fun import *

In [12]:
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

username = "jnwagwu"
api_key = "119wNvkoTSGwHIxORQ85"

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Import Data

In [13]:
def get_sub_dirs(path):
    sub_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return sub_dirs

root_path = os.getcwd()
opt_models_path= os.path.join(root_path,"opt_models")

In [14]:
get_sub_dirs(opt_models_path)

['light_0',
 'floor_2',
 'roof_2',
 'test_01',
 'shgc_b1',
 'window_u',
 'wall_b1',
 'floor_b1',
 'daylight_0',
 'roof_b1',
 'equip_0',
 'clgsetp_0',
 'roof_0',
 'htgsetp_0']

In [15]:
group_models = {
    "SHGC_and_True_Exteriors": ["shgc_b1", "roof_b1", "wall_b1", "floor_b1"],
    "Exteriors": ["wall_b1", "floor_2", "roof_0"],
    "Exteriors and Windows": ["wall_b1", "floor_2", "roof_0", "window_u", "shgc_b1"],
    "Electric Interior Gains": ["light_0", "equip_0"],
    "Setpoints": ["clgsetp_0", "htgsetp_0"],
    "Daylight": ["daylight_0"]

}

group_models["All"] = group_models["Exteriors and Windows"] + group_models["Electric Interior Gains"] + group_models["Setpoints"] + group_models["Daylight"]
print(len(group_models["All"]))


10


In [16]:
# labels 
labels = {
    "Wall R [ft2-ºF-hr/Btu]" : "wall",
    "Floor R [ft2-ºF-hr/Btu": "floor",
    "Roof R [ft2-ºF-hr/Btu": "roof",
    "SHGC": "shgc",
    "U-Factor [Btu/ft2-F-hr]": "window",
    "Light Power Density [W/ft2]": "light",
    "Equip. Power Density [W/ft2]": "equip",
    "Heating Setp. [ºF]": "htgsetp",
    "Cooling Setp. [ºF]": "clgsetp",
    "Daylight Controls": "daylight"

}
labels_rev = {v: k for k,v in labels.items()}
# labels_rev

In [24]:
# assign units 
groups  = {
    "r_val": ["wall", "floor", "roof"],
    "u_val": ["window"],
    "efficiency": ["light", "equip"],
    "setp": ["setp"],
    "none": ["shgc", "daylight"]
}

units = {
    "u_val": "W/m2-K",
    "r_val": "m2-K/W",
    "efficiency": "W/m2",
    "setp": "C",
    "none": ""
}

# https://betterhomesbc.ca/products/what-is-r-or-rsi-value/#:~:text=The%20R%2DValue%20is%20an,transfer%20through%20the%20insulating%20material.

r_val_si_to_imperial_factor = 5.678 # multiply the si r value by this number

def u_val_si_to_imperial(u_val):
    return u_val * (1/ r_val_si_to_imperial_factor )

def r_val_si_to_imperial(r_val):
    return r_val * r_val_si_to_imperial_factor 

def celcius_to_f(temp):
    return temp * (9/5) + 32

def w_m2_to_w_ft2(energy):
    return energy * (1/10.76)

units_imperial = {
    "u_val": ("Btu/ft2-F-hr", u_val_si_to_imperial ),
    "r_val": ("ft2-F-hr/Btu", r_val_si_to_imperial ),
    "efficiency": ("W/ft2", w_m2_to_w_ft2),
    "setp": ("F", celcius_to_f),
    "none": ("", lambda a: a),
}
keys = list(group_models["All"])
key_units = {}
for key in keys:
    # assign key to group 
    for k, v in groups.items():
        for string in v:
            if string in key:
                key_units[key] = (units[k], units_imperial[k])
# key_units

In [26]:
key_units

{'wall_b1': ('m2-K/W',
  ('ft2-F-hr/Btu', <function __main__.r_val_si_to_imperial(r_val)>)),
 'floor_2': ('m2-K/W',
  ('ft2-F-hr/Btu', <function __main__.r_val_si_to_imperial(r_val)>)),
 'roof_0': ('m2-K/W',
  ('ft2-F-hr/Btu', <function __main__.r_val_si_to_imperial(r_val)>)),
 'window_u': ('W/m2-K',
  ('Btu/ft2-F-hr', <function __main__.u_val_si_to_imperial(u_val)>)),
 'shgc_b1': ('', ('', <function __main__.<lambda>(a)>)),
 'light_0': ('W/m2', ('W/ft2', <function __main__.w_m2_to_w_ft2(energy)>)),
 'equip_0': ('W/m2', ('W/ft2', <function __main__.w_m2_to_w_ft2(energy)>)),
 'clgsetp_0': ('C', ('F', <function __main__.celcius_to_f(temp)>)),
 'htgsetp_0': ('C', ('F', <function __main__.celcius_to_f(temp)>)),
 'daylight_0': ('', ('', <function __main__.<lambda>(a)>))}

In [18]:
def get_group_csv(group_model):
    units_data = key_units[group_model]
    conversion_function = units_data[1][1]
    # print(units)
    group_model_path = os.path.join(opt_models_path, group_model )
    csv_path = os.path.join(group_model_path, "data.csv")
    df = pd.read_csv(csv_path, index_col="Unnamed: 0")

    # add imperial values 
    df["imperial_vals"] = conversion_function(df["vals"])
    df["imperial_vals_units"] = units_data[1][0]
    
    return df

In [19]:
def get_group(group_of_interest, example=False):
    group_data = {model: get_group_csv(model) for model in group_models[group_of_interest]}
    
    group_keys = list(group_data.keys())
    # print(group_keys)
    if example:
        first_data = list(group_data.keys())[0]
        display(group_data[first_data])

    
    # simple group keys
    simple_gk = [k.split("_")[0] for k in group_keys]

    # get labels 
    group_labels = {real_key: labels_rev[group_key] for group_key, real_key in zip(simple_gk, group_keys) }

        
    return group_keys, group_data, group_of_interest, group_labels

    # envelope 

# Exterior Surfaces

In [20]:
es_data = get_group("Exteriors", False)

In [21]:
four_compare(es_data[0], es_data[1], es_data[2], es_data[3])

Link  https://plotly.com/~jnwagwu/161/


In [22]:
correlations(es_data[1], es_data[2])

# SHGC + True Exteriors

In [27]:
# shgc = get_group("SHGC_and_True_Exteriors", False)
# data = shgc
# four_compare(data[0], data[1], data[2], data[3])
# correlations(data[1], data[2])


# Ext Surfaces + SHGV + U Window

In [28]:
data = get_group("Exteriors and Windows", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/163/


# Lighting, Equip

In [29]:
data = get_group("Electric Interior Gains", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/165/


# Setpoints

In [30]:
data = get_group("Setpoints", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/104/


# Daylight Controls

In [31]:
data = get_group("Daylight", False)
four_compare(data[0], data[1], data[2], data[3])
# correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/108/


# All

In [32]:
data = get_group("All", False)
four_compare(data[0], data[1], data[2], data[3])
correlations(data[1], data[2])

Link  https://plotly.com/~jnwagwu/112/


In [33]:
# todo bar graph with best case energy and equip savings for each 
# show feasible on this graph 
# graph with best case after feasibilty analysis 
# combinations 

In [34]:
# assign units 
groups  = {
    "r_val": ["wall", "floor", "roof"],
    "u_val": ["window"],
    "efficiency": ["light", "equip"],
    "setp": ["setp"],
    "none": ["shgc", "daylight"]
}

units = {
    "r_val": "W/m2-K",
    "u_val": "m2-K/W",
    "efficiency": "W/m2",
    "setp": "C",
    "none": ""
}
keys = data[0]
key_units = {}
for key in keys:
    # assign key to group 
    for k, v in groups.items():
        for string in v:
            if string in key:
                key_units[key] = units[k]

In [35]:
group_data = data[1]
group_data["clgsetp_0"]

,elect,hot,chill,elect_c,hot_c,chill_c,total,total_c,vals,perc_change,imperial_vals,imperial_vals_units
0.0,142453.408250,71662.765076,65269.397110,25328.67,9780.94,7794.79,279385.570436,42904.40,29.814815,0.0,85.666667,F
1.0,142294.173883,71564.224684,64225.513191,25275.21,9767.49,7670.13,278083.911757,42712.83,30.112963,1.0,86.203333,F
2.5,142104.495645,71428.555387,62787.338560,25203.02,9748.97,7498.38,276320.389593,42450.37,30.560185,2.5,87.008333,F
5.0,141899.559867,71234.462712,60784.825053,25123.38,9722.48,7259.23,273918.847632,42105.09,31.305556,5.0,88.350000,F
10.0,141737.930000,70957.400071,57896.626712,25035.55,9684.67,6914.30,270591.956783,41634.52,32.796296,10.0,91.033333,F
25.0,141696.002425,70845.621610,55974.825077,24994.49,9669.41,6684.79,268516.449112,41348.69,37.268519,25.0,99.083333,F
50.0,141696.002379,70845.621609,55974.816964,24994.49,9669.41,6684.79,268516.440953,41348.69,44.722222,50.0,112.500000,F
98.0,141696.002379,70845.621609,55974.816964,24994.49,9669.41,6684.79,268516.440953,41348.69,59.033333,98.0,138.260000,F
75.0,141696.002379,70845.621609,55974.816964,24994.49,9669.41,6684.79,268516.440953,41348.69,52.175926,75.0,125.916667,F


In [36]:

max_vals = {}
for ix, k in enumerate(group_data.keys()):
    x_ = group_data[k]["total_c"] - group_data[k].loc[0]["total_c"]
    z_ = group_data[k]["total"] - group_data[k].loc[0]["total"]
    y_ = list(group_data[k]["vals"])
    
    # concentrate on possibility of savings
    x = [-1*min(val, 0) for val in x_]
    z = [-1*min(val, 0) for val in z_]

    max_save = max(x)
    max_save_ix = x.index(max_save)
    max_save_val = y_[max_save_ix]
    max_save_energy = z[max_save_ix]
    max_vals[k] = (max_save, max_save_val, max_save_energy)
print(max_vals)

{'wall_b1': (331.45000000000437, 6.971830985915493, 2480.1814389269857), 'floor_2': (698.0999999999985, 0.0704225352112679, 5318.357948485063), 'roof_0': (496.77999999999156, 6.971830985915493, 3647.0607679850655), 'window_u': (1101.479999999996, 0.024, 8241.055381817569), 'shgc_b1': (634.1800000000003, 0.0061999999999999, 4893.969440043496), 'light_0': (5945.779999999999, 0.2647921962510597, 30749.175200569152), 'equip_0': (11057.04, 0.3999999999999986, 55328.06305150967), 'clgsetp_0': (1555.7099999999991, 37.26851851851858, 10869.121323386906), 'htgsetp_0': (5311.669999999998, 0.3844444444444453, 38874.54386635299), 'daylight_0': (1058.2099999999991, 1, 5700.413144911989)}


## Line Up

In [37]:
fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=(
            "Cost Savings [$]",
            "Feasible Cost Savings [$]",
            # "Energy Savings [$]",
            # "Feasible Energy "
        ))

x = keys
y = [val[0] for val in max_vals.values()]
text = [f"{np.round(v[1],2)} {key_units[k]}" for k,v in max_vals.items()]
fig.add_trace(go.Bar(x=x, y=y, text=text, textposition="auto"), row=1, col=1)
fig.show()

In [38]:
fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            # "Cost Savings [$]",
            # "Feasible Cost",
            "Energy Savings [$]",
            "Feasible Energy Savings [$]"
        ))

x = keys
y = [val[0] for val in max_vals.values()]
y2 = [val[2] for val in max_vals.values()]
# fig.add_trace(go.Bar(x=x, y=y, text=text, textposition="auto"), row=1, col=1)
fig.add_trace(go.Bar(x=x, y=y2, text=text, textposition="auto"), row=1, col=1)
fig.show()

In [39]:
# feasability analysis 

# Wall R-Value [units]
# Roof R-Value [units] 
# Floor R-Value [units]
# Solar Heat Gain Coefficient (SHGC)
# U-Factor [units]
# Equipment Watts per Zone Floor Area [units]
# Lighting Watts per Zone Floor Area [units]
# Cooling Setpoint [units]
# Heating Setpoint [units]
# Daylight Controls Applied

